In [16]:
import json
import os
import tarfile
import warnings

import numpy as np
import pandas as pd
import requests
import spacy
# import wget
# from elevate import elevate
from spacy.lang.en import English

import _pickle as pickle
from flask import Flask
# from flask_cors import CORS

import pandas as pd
from collections import Counter
import re
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer
import scattertext as st
from pprint import pprint

#Imports
from json import loads
from lxml import html
from requests import Session
import pandas as pd
from concurrent.futures import ThreadPoolExecutor as Executor
from itertools import count

In [23]:
warnings.filterwarnings('ignore')

df = pd.read_csv('C:\\Users\\lilyx\\BR-Data-Science\\test-version-v25\\trimmed_20k.csv')

lg_url = "https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz"

r = requests.get(lg_url, allow_redirects=True)
open('sm.zip', 'wb').write(r.content)

tar = tarfile.open('sm.zip', "r:gz")
tar.extractall('down_sm')

nlp = spacy.load("./down_sm/en_core_web_sm-2.1.0/en_core_web_sm/en_core_web_sm-2.1.0")
nlp.Defaults.stop_words |= {'check-in','=','= =','male','u','want', 'u want', 'cuz','him',"i've", 'deaf','on', 'her','told','told him','ins', '1 check','I', 'i"m', 'i', ' ', 'it', "it's", 'it.','they','coffee','place','they', 'the', 'this','its', 'l','-','they','this','don"t','the ', ' the', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.',','}


base_url = "https://www.yelp.com/biz/" # add business id
api_url = "/review_feed?sort_by=date_desc&start="
bid = "flower-child-addison-2" # business id

class Scraper():
    def __init__(self):
        self.data = pd.DataFrame()

    def get_data(self, n, bid=bid):
        with Session() as s:
            with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                r = loads(resp.content) #converts json response into a dictionary
                _html = html.fromstring(r['review_list']) #loads from dictionary

                dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                df = pd.DataFrame([dates, reviews, ratings]).T

                self.data = pd.concat([self.data,df])

    def scrape(self): #makes it faster
        # multithreaded looping
        with Executor(max_workers=40) as e:
            list(e.map(self.get_data, range(10)))


s = Scraper()
s.scrape()
df = s.data
df = df.sample(100)


corpus = st.CorpusFromPandas(df, 
                             category_col=2, 
                             text_col=1,
                             nlp=nlp).build()

term_freq_df = corpus.get_term_freq_df()
term_freq_df['highratingscore'] = corpus.get_scaled_f_scores('5.0 star rating')

term_freq_df['poorratingscore'] = corpus.get_scaled_f_scores('1.0 star rating')

df = term_freq_df.sort_values(by= 'poorratingscore', ascending = False)

df['highratingscore'] = round(df['highratingscore'], 2)
df['poorratingscore'] = round(df['poorratingscore'], 2)

df.to_json(r'termFrequenciesCondensed.json', orient='table')

In [22]:
#check if it's working
import json

df.to_json(orient='table')

'{"schema": {"fields":[{"name":"term","type":"string"},{"name":"5.0 star rating freq","type":"integer"},{"name":"1.0 star rating freq","type":"integer"},{"name":"2.0 star rating freq","type":"integer"},{"name":"4.0 star rating freq","type":"integer"},{"name":"3.0 star rating freq","type":"integer"},{"name":"highratingscore","type":"number"},{"name":"poorratingscore","type":"number"}],"primaryKey":["term"],"pandas_version":"0.20.0"}, "data": [{"term":"practices while","5.0 star rating freq":0,"1.0 star rating freq":1,"2.0 star rating freq":0,"4.0 star rating freq":0,"3.0 star rating freq":0,"highratingscore":0.46,"poorratingscore":1.0},{"term":"and\\/or changing","5.0 star rating freq":0,"1.0 star rating freq":1,"2.0 star rating freq":0,"4.0 star rating freq":0,"3.0 star rating freq":0,"highratingscore":0.46,"poorratingscore":1.0},{"term":"washing","5.0 star rating freq":0,"1.0 star rating freq":1,"2.0 star rating freq":0,"4.0 star rating freq":0,"3.0 star rating freq":0,"highratingscor